
This notebook implements the Entity Extarction by exploiting the spacy library. This is very much similar to AutoML (as we dont know what goes inside the model being used by spacy to make the entity prediction)

In [ ]:
import pandas as pd
import json
import spacy
from spacy import displacy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# variables to hold the data split type and text
documents = []
category = []

# variables to hold the labels info first entry 'null' is inserted already
# beacuse dictionary doesnt allow simple append if its an empty dictionary
placeholder = {'Date':['null'], 'Address': ['null'], 'Name':['null'], 'Amount':['null']}

# csv file has filename (including google cloud storage path) and data type, this is created by AutoML automatically. 
# column 2 has been added seaprately to give just the filename excluding 'path' in google storage

list_ = pd.read_csv(r'/content/drive/MyDrive/ING/Filtered data/text_extraction.csv',usecols=[0,2], names=['Type', 'Filename'], header=None)
for type_, file_name in list_.itertuples(index=False):
    # we will need encoding as data is stired in bytes format
    with open('/content/drive/MyDrive/ING/Filtered data/{}'.format(file_name), 'r', encoding="utf8") as json_file:
        json_list = list(json_file)

    for json_str in json_list:
        result = json.loads(json_str)
        documents.append(result['document']['documentText']['content'])
        category.append(type_)

        # data might be misisng some label, this variable will tell after completiong of first loop 
        # what is missing
        truth_table = {'Date': False, 'Address': False, 'Name': False, 'Amount':False}
    
        for i in range(len(result['annotations'])):
                # we are appending only the 'Offset' values now, we will extract the actual label using offset and text later
                placeholder[result['annotations'][i]['displayName']].append(result['annotations'][i]['textExtraction']['textSegment'])
                truth_table[result['annotations'][i]['displayName']] = True
        
        # the label which has flag 'False' in truth table need to append empty dictionary in placeholder
        for key, value in truth_table.items():
            if value == False:
                placeholder[key].append({})

# removing the first entry, see sbove for why           
placeholder['Date'] = placeholder['Date'][1:]
placeholder['Address']= placeholder['Address'][1:]
placeholder['Name'] = placeholder['Name'][1:]
placeholder['Amount'] = placeholder['Amount'][1:]

# merging the data_type and text with the placeholder dictionary
placeholder['text'] = documents
placeholder['data_type'] = category

In [ ]:
# check if length of all individual keys:values are same
print(placeholder.keys())
print(len(placeholder['Amount']))
print(len(placeholder['Date']))
print(len(placeholder['Address']))
print(len(placeholder['Name']))
print(len(placeholder['text']))
print(len(placeholder['data_type']))

dict_keys(['Date', 'Address', 'Name', 'Amount', 'text', 'data_type'])
949
949
949
949
949
949


In [ ]:
# convert dictionary to dataframe
data = pd.DataFrame.from_dict(placeholder)

In [ ]:
# extraction process for labels values using offset values and text
# first we get the start and end values for each label
for keys in ['Date', 'Address', 'Name', 'Amount']:
    # it can happen the word starts from begining, in that case OCR just gives endOffset and there is no startOffset
    # secondly when there is no label we had passsed emty dictioanry, which means it will have neither start and end
    # If conditions in lambda are to take care care of such edge cases
    data['{}_start'.format(keys)] = data[keys].apply(lambda x: x['startOffset'] if 'startOffset' in x.keys()
                        else ( 0 if 'endOffset' in x.keys() else None ))
                        
for keys in ['Date', 'Address', 'Name', 'Amount']:   
    data['{}_end'.format(keys)] = data[keys].apply(lambda x: x['endOffset'] 
                                                   if 'endOffset' in x.keys()   # in case of endoffset if only
                        else ( -1 if 'startOffset' in x.keys() else None))      # startoffset is present it means go till endoffset is end of text

In [ ]:
# a = [data.text[x:y] for x,y in data[['date_start', 'date_end']]]
# pd.set_option('display.max_rows', 1000)
# placeholder = {}

# finally we get the labels using start and end values and store it in placeholder
placeholder = {'Date':['null'], 'Address': ['null'], 'Name':['null'], 'Amount':['null']}

for keys in ['Date', 'Address', 'Name', 'Amount']: 

    for a,b,c in data[['{}_start'.format(keys), '{}_end'.format(keys), 'text']].itertuples(index=False):
        # print(a,b, type(a), type(b))
        if a != None and  b != None:
            placeholder[keys].append(c[int(a):int(b)])
        else:
            placeholder[keys].append(None)

placeholder['Date'] = placeholder['Date'][1:]
placeholder['Address']= placeholder['Address'][1:]
placeholder['Name'] = placeholder['Name'][1:]
placeholder['Amount'] = placeholder['Amount'][1:]

In [ ]:
# converting the dictionary to dataframe to be merged with main data in next step
df = pd.DataFrame.from_dict(placeholder)

# we dont need the offsets anymore
data.drop(['Date', 'Amount','Address','Name'], axis = 1, inplace= True)

In [ ]:
data

,text,data_type,Date_start,Address_start,Name_start,Amount_start,Date_end,Address_end,Name_end,Amount_end
0,"RESTAURANT SIN DU\nK3-113, JL IBRAHIM SULTAN\n...",TRAIN,97,18,0,183,107,67,17,190
1,อว)\nBECON STATIONER\nBecon Enterprise Sdn Bhd...,TRAIN,234,78,20,710,244,134,44,716
2,"GARDENIA BAKERIES (KL) SDN BHD (139386\nLot 3,...",TRAIN,212,39,0,654,222,91,30,658
3,MYDIN\nTRI SHAAS SDN BHD (728515-M)\nMYDIN MAR...,TRAIN,245,55,6,736,254,128,23,741
4,"99 SPEED MART S/B (519537-X)\nLOT P.T. 2811, J...",TRAIN,187,29,0,261,195,93,17,266
...,...,...,...,...,...,...,...,...,...,...
944,"SANYU STATIONERY SHOP\nNO. 316&33G, JALAN SETI...",TEST,492,22,0,386,502,78,21,390
945,"RESTORAN WAN SHENG\n002043319-W\nNo.2, Jalan T...",TEST,195,31,0,457,205,116,18,461
946,PAID\n1 MAR 2018\nMR. D.I.Y W) SDN BHD\n(CO Re...,TEST,5,55,16,476,15,150,36,481
947,S&Y STATIONERY\n(002050590-H)\nNO. 36G JALAN B...,TEST,331,29,0,586,342,122,14,591


In [ ]:
count_nan = data.isnull().sum()          # check for null values
count_nan

text             0
data_type        0
Date_start       3
Address_start    0
Name_start       2
Amount_start     5
Date_end         3
Address_end      0
Name_end         2
Amount_end       5
dtype: int64

In [ ]:
data.dropna(subset = ["Date_start"], inplace=True)     #   drop null values
data.dropna(subset = ["Name_start"], inplace=True)
data.dropna(subset = ["Amount_start"], inplace=True)
data.dropna(subset = ["Address_start"], inplace=True)
data.reset_index(drop=True, inplace=True)

In [ ]:
data.isnull().sum()     # no null values anymore

text             0
data_type        0
Date_start       0
Address_start    0
Name_start       0
Amount_start     0
Date_end         0
Address_end      0
Name_end         0
Amount_end       0
dtype: int64

In [ ]:
data = data.astype({"Date_start": int, "Address_start": int, "Name_start": int, "Amount_start": int, "Date_end": int, "Address_end": int, "Name_end": int, "Amount_end": int})

In [ ]:
type(data.iloc[0][7])    # conversion to integer type - successful

numpy.int64

In [ ]:
data

,text,data_type,Date_start,Address_start,Name_start,Amount_start,Date_end,Address_end,Name_end,Amount_end
0,"RESTAURANT SIN DU\nK3-113, JL IBRAHIM SULTAN\n...",TRAIN,97,18,0,183,107,67,17,190
1,อว)\nBECON STATIONER\nBecon Enterprise Sdn Bhd...,TRAIN,234,78,20,710,244,134,44,716
2,"GARDENIA BAKERIES (KL) SDN BHD (139386\nLot 3,...",TRAIN,212,39,0,654,222,91,30,658
3,MYDIN\nTRI SHAAS SDN BHD (728515-M)\nMYDIN MAR...,TRAIN,245,55,6,736,254,128,23,741
4,"99 SPEED MART S/B (519537-X)\nLOT P.T. 2811, J...",TRAIN,187,29,0,261,195,93,17,266
...,...,...,...,...,...,...,...,...,...,...
934,"SANYU STATIONERY SHOP\nNO. 316&33G, JALAN SETI...",TEST,492,22,0,386,502,78,21,390
935,"RESTORAN WAN SHENG\n002043319-W\nNo.2, Jalan T...",TEST,195,31,0,457,205,116,18,461
936,PAID\n1 MAR 2018\nMR. D.I.Y W) SDN BHD\n(CO Re...,TEST,5,55,16,476,15,150,36,481
937,S&Y STATIONERY\n(002050590-H)\nNO. 36G JALAN B...,TEST,331,29,0,586,342,122,14,591


## Train and test dataset

In [ ]:
data_train = data[data['data_type']=='TRAIN']       # choose data only for training
data_train = data_train.reset_index(drop=True)
data_train

,text,data_type,Date_start,Address_start,Name_start,Amount_start,Date_end,Address_end,Name_end,Amount_end
0,"RESTAURANT SIN DU\nK3-113, JL IBRAHIM SULTAN\n...",TRAIN,97,18,0,183,107,67,17,190
1,อว)\nBECON STATIONER\nBecon Enterprise Sdn Bhd...,TRAIN,234,78,20,710,244,134,44,716
2,"GARDENIA BAKERIES (KL) SDN BHD (139386\nLot 3,...",TRAIN,212,39,0,654,222,91,30,658
3,MYDIN\nTRI SHAAS SDN BHD (728515-M)\nMYDIN MAR...,TRAIN,245,55,6,736,254,128,23,741
4,"99 SPEED MART S/B (519537-X)\nLOT P.T. 2811, J...",TRAIN,187,29,0,261,195,93,17,266
...,...,...,...,...,...,...,...,...,...,...
745,Cum\nAMANO MALAYSIA SON BED (682288-V)\n12 JAL...,TRAIN,239,38,4,390,249,111,26,394
746,"Y SOON FATT S/B (81497-P)\nLun 1504, BATU 8 1/...",TRAIN,635,26,0,521,643,98,15,527
747,"SANYO STATIONERY SHOP\nNO. 316833G, JALAN SETI...",TRAIN,487,22,0,382,497,77,21,386
748,MR. D.I.Y. (M) SDN BHD\nCo-REG:860671-D\nLOT 1...,TRAIN,794,39,0,738,802,135,22,743


In [ ]:
data_test = data[data['data_type']=='TEST']       # choose data only for training
data_test = data_test.reset_index(drop=True)
data_test

,text,data_type,Date_start,Address_start,Name_start,Amount_start,Date_end,Address_end,Name_end,Amount_end
0,POPULAR BOOK\nCO. (M) SDN BHD\n(Co. No. 113825...,TEST,266,75,0,519,274,138,28,524
1,"POSTED\noff)\nBEMED (SP) SDN. BHD.\nN0.49, Jal...",TEST,171,33,12,466,182,104,31,472
2,"SANYU STATIONERY SHOP\nNO. 316833G, JALAN SETI...",TEST,497,22,0,392,507,77,21,396
3,ECONSAVE OF\nOE\nECONSAVE CASH & CARRY (FC) $/...,TEST,145,57,15,367,153,122,45,371
4,PASARAYA BORONG PINTAR\nSDN BHD\nBR NO.: (1245...,TEST,138,50,0,342,148,114,30,346
...,...,...,...,...,...,...,...,...,...,...
91,"SANYU STATIONERY SHOP\nNO. 316&33G, JALAN SETI...",TEST,492,22,0,386,502,78,21,390
92,"RESTORAN WAN SHENG\n002043319-W\nNo.2, Jalan T...",TEST,195,31,0,457,205,116,18,461
93,PAID\n1 MAR 2018\nMR. D.I.Y W) SDN BHD\n(CO Re...,TEST,5,55,16,476,15,150,36,481
94,S&Y STATIONERY\n(002050590-H)\nNO. 36G JALAN B...,TEST,331,29,0,586,342,122,14,591


## Create one traning dataset (in Spacy specif required format)

In [ ]:
TRAIN_DATA_MULTI = []
entity_name_train = ['DATE', 'ADDRESS', 'NAME', 'AMOUNT'] 

for i in range(len(data_train)):
  for j in range(1): 
    dicts={}
    dicts['entities']= [(data_train.iloc[i][2+j],data_train.iloc[i][6+j],entity_name_train[j]), 
                        (data_train.iloc[i][3+j],data_train.iloc[i][7+j],entity_name_train[j+1]), 
                        (data_train.iloc[i][4+j],data_train.iloc[i][8+j],entity_name_train[j+2]), 
                        (data_train.iloc[i][5+j],data_train.iloc[i][9+j],entity_name_train[j+3])]
    TRAIN_DATA_MULTI.append((data_train.text[i],dicts))

In [ ]:
TRAIN_DATA_MULTI[47][1]

{'entities': [(215, 225, 'DATE'),
  (42, 94, 'ADDRESS'),
  (0, 30, 'NAME'),
  (665, 670, 'AMOUNT')]}

## Create one test dataset (in Spacy specif required format)

In [ ]:
data_test

,text,data_type,Date_start,Address_start,Name_start,Amount_start,Date_end,Address_end,Name_end,Amount_end
0,POPULAR BOOK\nCO. (M) SDN BHD\n(Co. No. 113825...,TEST,266,75,0,519,274,138,28,524
1,"POSTED\noff)\nBEMED (SP) SDN. BHD.\nN0.49, Jal...",TEST,171,33,12,466,182,104,31,472
2,"SANYU STATIONERY SHOP\nNO. 316833G, JALAN SETI...",TEST,497,22,0,392,507,77,21,396
3,ECONSAVE OF\nOE\nECONSAVE CASH & CARRY (FC) $/...,TEST,145,57,15,367,153,122,45,371
4,PASARAYA BORONG PINTAR\nSDN BHD\nBR NO.: (1245...,TEST,138,50,0,342,148,114,30,346
...,...,...,...,...,...,...,...,...,...,...
91,"SANYU STATIONERY SHOP\nNO. 316&33G, JALAN SETI...",TEST,492,22,0,386,502,78,21,390
92,"RESTORAN WAN SHENG\n002043319-W\nNo.2, Jalan T...",TEST,195,31,0,457,205,116,18,461
93,PAID\n1 MAR 2018\nMR. D.I.Y W) SDN BHD\n(CO Re...,TEST,5,55,16,476,15,150,36,481
94,S&Y STATIONERY\n(002050590-H)\nNO. 36G JALAN B...,TEST,331,29,0,586,342,122,14,591


In [ ]:
TEST_DATA_MULTI = []
entity_name_test = ['DATE', 'ADDRESS', 'NAME', 'AMOUNT'] 

for i in range(len(data_test)):
  for j in range(1): 
    dicts={}
    dicts['entities']= [(data_test.iloc[i][2+j],data_test.iloc[i][6+j],entity_name_test[j]), 
                        (data_test.iloc[i][3+j],data_test.iloc[i][7+j],entity_name_test[j+1]), 
                        (data_test.iloc[i][4+j],data_test.iloc[i][8+j],entity_name_test[j+2]), 
                        (data_test.iloc[i][5+j],data_test.iloc[i][9+j],entity_name_test[j+3])]
    TEST_DATA_MULTI.append((data_test.text[i],dicts))

In [ ]:
TEST_DATA_MULTI[47][1]

{'entities': [(415, 423, 'DATE'),
  (44, 108, 'ADDRESS'),
  (0, 20, 'NAME'),
  (363, 368, 'AMOUNT')]}

## Training NER

In [ ]:
import warnings

In [ ]:
from spacy.util import minibatch, compounding
import random
import datetime as dt

In [ ]:
# Import and load the spacy model
nlp=spacy.load("en_core_web_sm") 

# add NER to the pipeline and the new label
ner = nlp.get_pipe("ner")

labels = ["DATE", "ADDRESS", "NAME", "AMOUNT"]
for label in labels:
    ner.add_label(label)


# get the names of the components we want to disable during training
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# start the training loop, only training NER
epochs = 30
optimizer = nlp.resume_training()
with nlp.disable_pipes(*other_pipes), warnings.catch_warnings():
    warnings.filterwarnings("once", category=UserWarning, module='spacy')
    sizes = compounding(1.0, 32.0, 1.001) # second element is a batch size, the last element is the compounding rate
    
    # batch up the examples using spaCy's minibatch
    for epoch in range(epochs):
        examples = TRAIN_DATA_MULTI
        random.shuffle(examples)
        batches = minibatch(examples, size=sizes)
        losses = {}
        
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.08, losses=losses)

        print("Losses ({}/{})".format(epoch + 1, epochs), losses)

In [ ]:
test_text = list(nlp.pipe(data_test.text))          
displacy.render(test_text, style='ent', jupyter=True, options={'distance': 90})    # check if model detects entities on the test set

In [ ]:
# Output directory
from pathlib import Path
output_dir=Path('/content/')

# Saving the model to the output directory
if not output_dir.exists():
  output_dir.mkdir()
nlp.meta['name'] = 'my_ner'  # rename model
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to /content


## Evaluate the model

In [ ]:
# Importing necessary libraries
from spacy.scorer import Scorer
from spacy.gold import GoldParse

In [ ]:
  # GoldParse: Module that parses the saved format accepted by the model
  # It creates an instance of the GoldParse Class to provide training examples to the entity recognizer
  def evaluate(examples):
    # test the saved model
    ner_model = spacy.load(output_dir)

    # Creating default scoring pipeline

    scorer = Scorer()
    try:
        for input_, annot in examples:
            doc_gold_text = ner_model.make_doc(input_)
            gold = GoldParse(doc_gold_text, entities=annot['entities'])
            pred_value = ner_model(input_)
            scorer.score(pred_value, gold)
    except Exception as e: print(e)

    print(scorer.scores)

In [ ]:
# Outputs precision, recall, f1 for train data
examples1 = TRAIN_DATA_MULTI
evaluate(examples1)
# Outputs precision, recall, f1 for tets data
examples = TEST_DATA_MULTI
evaluate(examples)

In [ ]:
examples1 = TRAIN_DATA_MULTI
evaluate(examples1)

{'uas': 0.0, 'las': 0.0, 'las_per_type': {'compound': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'root': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'npadvmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'intj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'dep': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nsubj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nummod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'appos': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'meta': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'dobj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'advmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'cc': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'acl': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'neg': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'acomp': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'prep': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'pobj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'conj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'quantmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'attr': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'amod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'det': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'poss': {'p': 0.0, 'r': 0